<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Kriptovaliutos ir rinkos
> Kriptovaliutų koreliacija su rinkomis - BTC, ETH, XRP su SP500, XAU. 

- toc: true 
- badges: true
- comments: false
- categories: [Dominykas Tautkus]
- image: images/chart-preview.png

In [1]:
#hide
%load_ext autoreload
%autoreload 2
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from scripts.read_data import read_api
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML
import altair as alt

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
#hide
df_all = read_api()
del df_all['LSK']
del df_all['EOS']
del df_all['NEO']
del df_all['TRX']
del df_all['ADA']
df_norm = (df_all - df_all.mean()) / (df_all.max() - df_all.min())

/home/runner/work/corr-app/corr-app/_notebooks/scripts/read_data.py:187: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [ ]:
#hide
from IPython.display import HTML

## Kriptovaliutų su rinkomis koreliacijos matrica

In [ ]:
#hide_input
sum_corr = df_all.corr().sum().sort_values(ascending=True).index.values
df_all[sum_corr].corr()
plt.figure(figsize=(15, 10))
sns.heatmap(df_all[sum_corr].corr(), annot=True, cmap="BrBG");


## Normalizuotos kriptovaliutų su rinkomis kainos 
> (Pasikrovimas gali užtrukti kelias sek.)

> Dešinėje galimas kelių kriptovaliutų pasirinkimas

In [ ]:
#hide_input 

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df_norm.index),
               y=list(df_norm.XAU),
               name="XAU",
               line=dict(color="Crimson")))


fig.add_trace(
    go.Scatter(x=list(df_norm.index),
               y=list(df_norm.XRP),
               name="XRP",
               line=dict(color="RebeccaPurple")))

fig.add_trace(
    go.Scatter(x=list(df_norm.index),
               y=list(df_norm.BTC),
               name="BTC",
               line=dict(color="Black")))

fig.add_trace(
    go.Scatter(x=list(df_norm.index),
               y=list(df_norm.ETH),
               name="ETH",
               line=dict(color="Orange")))

fig.add_trace(
    go.Scatter(x=list(df_norm.index),
               y=list(df_norm.SP500),
               name="SP500",
               line=dict(color="Green")))

fig.update_layout(
       width=900,
        height=900,
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            active=5,
#             x=1.57,
#             y=1.2,
            buttons=list([
                dict(label="XAU",
                     method="update",
                     args=[{"visible": [True, False, False, False, False]},
                           {"title": "XAU",
                            "annotations": []}]),
                dict(label="XRP",
                     method="update",
                     args=[{"visible": [False, True, False, False, False]},
                           {"title": "XRP",
                            "annotations": [] }]),
                 dict(label="BTC",
                     method="update",
                     args=[{"visible": [False, False, True, False, False]},
                           {"title": "BTC",
                            "annotations": [] }]),
                 dict(label="ETH",
                     method="update",
                     args=[{"visible": [False, False, False, True, False]},
                           {"title": "ETH",
                            "annotations": [] }]),
                dict(label="SP500",
                     method="update",
                     args=[{"visible": [False, False, False, False, True]},
                           {"title": "SP500",
                            "annotations": [] }]),
                dict(label="ALL",
                     method="update",
                     args=[{"visible": [True, True, True, True, True]},
                           {"title": "all",
                            "annotations": [] }]),
            ]),
        )
    ])

# Set title
fig.update_layout(
    #title_text="Normalizuotos kriptovaliutų su rinkomis kainos",
#     xaxis_domain=[0.20, 1.0]
    xaxis_domain=[1.0, 1.0])

HTML(fig.to_html()) 

In [ ]:
#hide
long_form = df_all.reset_index().melt('date', var_name='currency', value_name='price') 
df_partial = long_form[long_form.currency.isin(['ETH', 'BTC', 'SP500', 'XAU'])]
df_partial["date"] = pd.to_datetime(df_partial["date"])

## Kainos pagal laiko diapazonus

> (Laiko diapazono pasirinkimas apačioje pažymėjus norimą laikotarpį)

In [ ]:
#hide_input
brush = alt.selection(type='interval', encodings=['x'])
selection = alt.selection_multi(fields=['currency'], bind="legend")

base = alt.Chart(df_partial).mark_line().encode(
    x = 'date:T',
    y = 'price:Q',
    color = 'currency'
).properties(
    width = 700, 
    height = 625
)

upper = base.encode(
    alt.X('date:T', scale = alt.Scale(domain=brush)) 
    
)

lower=base.properties(
    height=70
).add_selection(brush) 

upper & lower 